In [1]:
#
# David Laziuk
# Decision Tree from scratch
#

In [2]:
#Imports
import math
import numpy as np
import pandas as pd
#Loading Data
data=pd.read_csv('breast-cancer-wisconsin.csv',na_values= "?")
#Dropping rows with NAs
data=data.dropna()
#Fixing value type
data['F6']=data['F6'].astype(int)
#Dropping Sample ID Column
data=data.iloc[:,1:]
#Shuffling Entries
data=data.sample(frac=1).reset_index(drop=True)
#70/30 split into train/test data
train_size=int(np.floor(data.shape[0]*.7))
features=(data.shape[1]-1)
X_train=data.iloc[0:train_size,0:features].reset_index(drop=True)
X_test=data.iloc[train_size:,0:features].reset_index(drop=True)
y_train=data.iloc[0:train_size,features].reset_index(drop=True)
y_test=data.iloc[train_size:,features].reset_index(drop=True)
del features,train_size

In [3]:
#Function to get Gini Impurity of Split
def get_impurity(X,y,col,val,terminate):
    #Store predicted class if clean split
    lPred=-1
    rPred=-1
    #Store all sample indexes that satisfy & fail given condition
    idxL=X[X[col]<=val].index
    idxR=X[X[col]>val].index
    #Class composition of samples after split
    compL=y[idxL].value_counts()
    compR=y[idxR].value_counts()
    #LEFT SPLIT: Condition Satisfied
    #Case 1: All samples satisfy condition, max impurity returned
    # ~Impurity of an empty set is undefined, but 1 is chosen as this split is worthless
    if(len(compL)==0):
        return 1,lPred,rPred
    #Case 2: Samples satisfying condition are all same class, lowest impurity (0)
    #Store Class to be returned as perfect split
    elif(len(compL)==1):
        leftS=compL.iloc[0]
        leftI=0
        lPred=compL.index[0]
    #Case 3: Samples satisying condition have mixed class, calculate impurity
    else:
        leftS=(compL[2]+compL[4])
        #Probability of each class
        p2L=compL[2]/leftS
        p4L=compL[4]/leftS
        #Impurity = 1-(P(c1)^2+P(c2)^2)
        leftI=1-((p2L**2)+(p4L**2))
        #If max_depth is reached, predict majority class and end branch
        #This is not done for Case 1/2 as those cases end branch naturally 
        if(terminate):
            if(compL[2]>compL[4]):
                lPred=2
            #Predict malignant if even, FP is prefered in reality (Could be changed to random)
            else:
                lPred=4
    #RIGHT SPLIT: Condition not satisfied
    #Repeat cases 1-3 identically
    if(len(compR)==0):
        return 1,lPred,rPred
    elif(len(compR)==1):
        rightS=compR.iloc[0]
        rightI=0
        rPred=compR.index[0]
    else:
        rightS=(compR[2]+compR[4])
        p2R=compR[2]/rightS
        p4R=compR[4]/rightS
        rightI=1-((p2R**2)+(p4R**2))
        if(terminate):
            if(compR[2]>compR[4]):
                rPred=2
            else:
                rPred=4
    #Size of parent node
    nodeS=leftS+rightS
    #Parent Node Impurity= Weighted Average of child impurities
    nodeI=((leftS/nodeS)*leftI)+((rightS/nodeS)*rightI)
    return nodeI,lPred,rPred

In [4]:
#Find find condition(feature<=value) with lowest Gini Impurity for data
def get_split(X,y,terminate):
    #Best Impurity,Column/Feature,Value  If perfect split: Best Left/Right class prediction
    bestI=1.1
    bestC=''
    bestV=-1
    bestL=-1
    bestR=-1
    #Check each Feature
    for col in list(X.columns):
        #Find all values contained in feature
        comp=sorted(list(X[col].value_counts().index))
        #Test all values (except max val, nothing greater)
        for val in comp:
            if(val!=10):
                #Get Impurity of Condition
                I,lPred,rPred=get_impurity(X,y,col,val,terminate)
                #Store if Lowest
                if(I<bestI):
                    bestI=I
                    bestC=col
                    bestV=val
                    bestL=lPred
                    bestR=rPred
    return bestI,bestC,bestV,bestL,bestR

In [5]:
#Node class: Contains col, cond, path & Leaf(Class if perfect split)
class Node:
    def __init__(self,Col,Val,Path,Leaf):
        self.Col=Col
        self.Val=Val
        self.Path=Path
        self.Leaf=Leaf

In [6]:
#Recursive function to fit CART decision tree 
def growTree(X,y,path,max_depth):
    #Terminate if max_depth reached
    terminate=False
    if(len(path)>=max_depth-1):
        terminate=True
    #Get best split for data
    I,col,val,lPred,rPred=get_split(X,y,terminate)
    #Add parent node to list, Leaf will be None as function will not be called for data of same class
    nodes.append(Node(col,val,path,None))
    #print("Node"+str(len(nodes))+": "+path)
    #print(col+'<='+str(val)+' left:'+str(lPred)+' right:'+str(rPred)+' Impurity: '+str(round(I,3)))
    #Child Node Operations
    #If left node is leaf:
    if(lPred!=-1):
        #Add leaf node to list with path+'l'
        nodes.append(Node(None,None,path+'l',lPred))
        #print("Node"+str(len(nodes))+": "+path+'l')
        #print('LEAF: '+str(lPred))
    else:
        #Reset indexes of data
        X=X.reset_index(drop=True)
        y=y.reset_index(drop=True)
        #Find indexes of samples satisfying condition
        idxL=X[X[col]<=val].index
        #Pass data of samples satisfying condition to be split, with path+'l'
        Xl=X.iloc[idxL]
        yl=y.iloc[idxL]
        growTree(Xl,yl,path+'l',max_depth)
    #Same operations for samples failing condition (right), with path+'r'
    if(rPred!=-1):
        nodes.append(Node(None,None,path+'r',rPred))
        #print("Node"+str(len(nodes))+": "+path+'r')
        #print('LEAF: '+str(rPred))
    else:
        X=X.reset_index(drop=True)
        y=y.reset_index(drop=True)
        idxR=X[X[col]>val].index
        Xr=X.iloc[idxR]
        yr=y.iloc[idxR]
        growTree(Xr,yr,path+'r',max_depth)

In [7]:
#Function to draw tree
def drawTree(ns):
    #Find depth, not necessarily=max_depth
    depth=-1
    for n in ns:
        if(len(n.Path)>depth):
            depth=len(n.Path)
    #Get full tree with all possible paths (for formatting)
    trash,full=fullTree([],'',depth)   
    #Iterate through all layers
    for l in range(depth+1):
        spacing=(7*(2**(depth-l)))#Exponentially increasing spacing based on height of layer
        #Find all nodes in layer based on path length
        paths=[]
        for n in ns:
            if(len(n.Path)==l):
                paths.append(n.Path)
        #Print first node individually
        if(l==0):
            temp='('+ns[0].Col+'<='+str(ns[0].Val)+')'
            print(temp.center(spacing),end='')
        #Find all possible paths for layer
        pps=[]
        for f in full:
            if(len(f)==l):
                pps.append(f)         
        #Main Print
        for p in pps:#Check all possible paths in layer
            if p in paths:#If path exists proceed
                for i in range(len(ns)):#Search through all nodes
                    if(ns[i].Path==p):#If path matches proceed
                        if(ns[i].Leaf==None):#If not leaf print condition
                            temp='('+ns[i].Col+'<='+str(ns[i].Val)+')'
                            print(temp.center(spacing),end='')
                        else:#If leaf print prediction
                            temp='(  '+str(ns[i].Leaf)+'  )'
                            print(temp.center(spacing),end='')
            #Else print spacing
            else:
                print('       '.center(spacing),end='')
        print()#End line
#Recursive method of getting all possible paths in a tree of set depth
def fullTree(paths,path,depth):
    if(len(path)<depth):
        p1,d=fullTree(paths,path+'l',depth)
        paths.append(p1)
        p2,d=fullTree(paths,path+'r',depth)
        paths.append(p2)
    return path,paths

In [8]:
#Growing Tree with original train data, max_depth=4 (not counting top node)
nodes=[]
growTree(X_train,y_train,"",4)
drawTree(nodes)#draw tree

                                                    (F3<=3)                                                     
                        (F6<=5)                                                 (F2<=1)                         
          (F2<=3)                     (F1<=1)                     (  2  )                     (F2<=4)           
   (F8<=8)       (F1<=4)       (  2  )       (F7<=3)                                   (F6<=3)       (F4<=1)    
(  2  )(  4  )(  2  )(  4  )              (  4  )(  4  )                            (  2  )(  4  )(  4  )(  4  )


In [9]:
#Custom Testing
acc=0
#Iterate through test samples
for i in range(X_test.shape[0]):
    #Extract truth
    truth=y_test.iloc[i]
    pred=0
    path=''#Start with empty path (top node)
    #Loop until prediction(leaf) is found
    while(pred==0):
        #Linear Search for node satisfying path
        #This is very unoptimized for readability
        #Optimal solution would be for Node Class to reference parent/children directly
        pos=-1
        for p in range(len(nodes)):
            if(nodes[p].Path==path):
                pos=p
                break
        #If node satisfying path is leaf, store prediction
        if(nodes[pos].Leaf!=None):
            pred=nodes[pos].Leaf
            #acc++ if pred is true
            if(pred==truth):
                acc+=1
        #If node satisying path is condition, test
        else:
            #Path+'l' if satisfied, +'r' otherwise, go deeper
            if(X_test.loc[i,nodes[pos].Col]<=nodes[pos].Val):
                path+="l"
            else:
                path+="r"
print('Accuracy: '+str(acc/y_test.shape[0]))

Accuracy: 0.9512195121951219
